In [1]:
import cv2
from ultralytics import YOLO

In [5]:
class GarbageDetector:
    def __init__(self,model_path,camera):
        self.model=YOLO(model_path)
        self.camera=camera
        self.thres_box_frame_ratio=0.6
        self.results=[]
        self.frame_size=1

    def remove_large(self):
        keep_boxes = []

        for i, box in enumerate(self.results[0].boxes):
            box_size=self.get_box_size(box)

            box_frame_ratio=box_size/self.frame_size
            
            if (box_frame_ratio<=self.thres_box_frame_ratio):
                keep_boxes.append(i)

        if keep_boxes:
            self.results[0].boxes = self.results[0].boxes[keep_boxes] 
        else:
            self.results[0].boxes = None

    def display_box_details(self):
        if self.results[0].boxes is not None:
            for box in self.results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                cls_id = int(box.cls[0])
                confidence = box.conf[0]
                class_name = self.results[0].names[cls_id]
                print(f"Class: {class_name}, Confidence: {confidence:.2f}, Box: [{x1}, {y1}, {x2}, {y2}]")

    def get_box_size(self,box):
        x1, y1, x2, y2 = box.xyxy[0]
        box_width=(x1-x2)
        box_height=(y1-y2)
        box_size=(box_width**2+box_height**2)**0.5
        return box_size

    def get_closest_object_info(self):
        if self.results[0].boxes is not None:
            closest_object=self.results[0].boxes[0]
            for box in self.results[0].boxes:
                if self.get_box_size(box)>self.get_box_size(closest_object):
                    closest_object=box
        box_info={}
        cls_id = int(box.cls[0])
        class_name = self.results[0].names[cls_id]
        box_info['Class']=class_name
        x1,y1,x2,y2=box.xyxy[0]
        box_info['Coordinates']=(float(x1),float(y1),float(x2),float(y2))
        
        if (class_name=="METAL" or class_name=="GLASS" or class_name=="PLASTIC"):
            box_info['isBiodegradable']=False
        else:
            box_info['isBiodegradable']=True

        return box_info
    
    def detect(self,ahead,thres_box_frame_ratio=0.6):
        self.thres_box_frame_ratio=thres_box_frame_ratio
        
        cap=cv2.VideoCapture(self.camera)

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        self.frame_size=(frame_width**2+frame_height**2)**0.5
        
        if not cap.isOpened():
            print("Error: Could not open camera.")
            exit()
        found=False
        while (not found):
            ret,frame=cap.read()
            if not ret:
                print("Error: Could not receive frame.")
                break
            self.results=self.model(frame,verbose=False)

            if ahead:
                self.remove_large()

            #self.display_box_details()
            
            annotated_frame= self.results[0].plot()
            cv2.imshow("Garbage Detection",annotated_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            if self.results[0].boxes is not None:
                found=True

        cap.release()
        cv2.destroyAllWindows()

        return self.get_closest_object_info()

    def pause(self):
        signal=False #get signal from arduino
        while True:
            if (signal):
                break
                
    def run(self):
        while True:
            try:
                box_info = self.detect(ahead=True)
                print(box_info)
                #send box_info to Arduino, move towards object
                #looking for signal
                #self.pause()
                box_info = self.detect(ahead=False)
                print(box_info)
                #self.pause() #collect object
            except:
                break
        



In [6]:
detector=GarbageDetector("runs/detect/train/weights/best_openvino_model",0)

In [9]:
detector.run()